In [1]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
tf.set_random_seed(1)
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
import matplotlib.pyplot as plt
import PIL.Image as Image
import math
import numpy as np
import tensorflow.keras.backend as K
from tensorflow.keras.utils import plot_model
import json
from tensorflow.keras.losses import mse
import os
import glob
import random
import sys
ros_path = '/opt/ros/kinetic/lib/python2.7/dist-packages'

if ros_path in sys.path:

    sys.path.remove(ros_path)

import cv2

sys.path.append('/opt/ros/kinetic/lib/python2.7/dist-packages')

from sklearn.model_selection import train_test_split
from tensorflow.keras.applications.inception_v3 import InceptionV3
import datetime
from tensorflow.keras.layers import Dense, Input, concatenate, Conv2D, MaxPooling2D, Flatten, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, CSVLogger, ReduceLROnPlateau
from tensorflow.keras.callbacks import TensorBoard
%matplotlib inline

Instructions for updating:
non-resource variables are not supported in the long term


# Generated Data

# Town 1

In [2]:
data_path = "/home/talha/Documents/New_Carla_Dataset/Town1/"

new_size = (112, 112) 

Left_images = []
Right_images = []
Disparity = []
Tx = []
Ty = []
Tz = []
Baseline = []
Pitch = []
Fx = []
Fy = []
U0 = []
V0 = []
X = []
Y = []
Z = []


for folder in os.listdir(data_path):
    
    if "episode" in folder:
    
        episode_data = os.listdir(data_path+folder)
        
        file = open(data_path+folder+"/params.txt", "r")

        i = 0

        for line in file:
    
            if ":" in line:
        
                if i == 0:
        
                    fov = float(line.split(":")[1])
            
                elif i == 1:
            
                    x = float(line.split(":")[1])
            
                elif i == 2:
            
                    y = float(line.split(":")[1])
            
                elif i == 3:
            
                    z = float(line.split(":")[1])
            
                elif i == 4:
            
                    p = float(line.split(":")[1])

            
            i += 1
        
        
        for fname in episode_data:
        
            if "Left" in fname:
                
                l_im = cv2.imread(data_path+folder+"/"+fname, 0)
                r_im = cv2.imread(data_path+folder+"/"+'RightRGB_'+fname.split("_")[1], 0)
                stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
                disparity_map = stereo.compute(l_im, r_im)
                disparity_value = np.mean(disparity_map)
        
                l_im = cv2.imread(data_path+folder+"/"+fname)
                r_im = cv2.imread(data_path+folder+"/"+'RightRGB_'+fname.split("_")[1])
        
                Left_images.append(l_im)
                Right_images.append(r_im)
                Disparity.append(disparity_value)
                Tx.append(x)
                Ty.append(y)
                Tz.append(z)
                Baseline.append(x)
                Pitch.append(p)
                focal_length = new_size[0]/(2*np.tan(fov*np.pi/360))
                center_x = new_size[0]/2
                center_y = center_x
                Fx.append(focal_length)
                Fy.append(focal_length)
                U0.append(center_x)
                V0.append(center_y)
                
                xCam = (focal_length * x) / disparity_value
                yCam = - (xCam / focal_length) * (56 - center_x)
                zCam = (xCam / focal_length) * (center_y - 56)

                yWorld = yCam + y
                xWorld = xCam * math.cos(p) + zCam*math.sin(p) + x
                zWorld = - xCam * math.sin(p) + zCam*math.cos(p) + z
        
                X.append(xWorld)
                Y.append(yWorld)
                Z.append(zWorld)
        




In [3]:
np.shape(X)

(58596,)

# Town 2

In [4]:
data_path = "/home/talha/Documents/New_Carla_Dataset/Town2/"

new_size = (112, 112) 


for folder in os.listdir(data_path):
    
    if "episode" in folder:
    
        episode_data = os.listdir(data_path+folder)
        
        file = open(data_path+folder+"/params.txt", "r")

        i = 0

        for line in file:
    
            if ":" in line:
        
                if i == 0:
        
                    fov = float(line.split(":")[1])
            
                elif i == 1:
            
                    x = float(line.split(":")[1])
            
                elif i == 2:
            
                    y = float(line.split(":")[1])
            
                elif i == 3:
            
                    z = float(line.split(":")[1])
            
                elif i == 4:
            
                    p = float(line.split(":")[1])
            
            i += 1
        
        
        for fname in episode_data:
        
            if "Left" in fname:
                
                l_im = cv2.imread(data_path+folder+"/"+fname, 0)
                r_im = cv2.imread(data_path+folder+"/"+'RightRGB_'+fname.split("_")[1], 0)
                stereo = cv2.StereoBM_create(numDisparities=16, blockSize=15)
                disparity_map = stereo.compute(l_im, r_im)
                disparity_value = np.mean(disparity_map)
        
                l_im = cv2.imread(data_path+folder+"/"+fname)
                r_im = cv2.imread(data_path+folder+"/"+'RightRGB_'+fname.split("_")[1])
        
                Left_images.append(l_im)
                Right_images.append(r_im)
                Disparity.append(disparity_value)
                Tx.append(x)
                Ty.append(y)
                Tz.append(z)
                Baseline.append(x)
                Pitch.append(p)
                focal_length = new_size[0]/(2*np.tan(fov*np.pi/360))
                center_x = new_size[0]/2
                center_y = center_x
                Fx.append(focal_length)
                Fy.append(focal_length)
                U0.append(center_x)
                V0.append(center_y)
                
                xCam = (focal_length * x) / disparity_value
                yCam = - (xCam / focal_length) * (56 - center_x)
                zCam = (xCam / focal_length) * (center_y - 56)

                yWorld = yCam + y
                xWorld = xCam * math.cos(p) + zCam*math.sin(p) + x
                zWorld = - xCam * math.sin(p) + zCam*math.cos(p) + z
        
                X.append(xWorld)
                Y.append(yWorld)
                Z.append(zWorld)
        
        

In [5]:
print ("Training Dataset: ",len(Left_images)*0.7, "Test Dataset: ", len(Left_images)*0.3)

Training Dataset:  86111.9 Test Dataset:  36905.1


In [6]:
print ("Training Dataset: ",len(Left_images)*0.7, "Test Dataset: ", len(Left_images)*0.3)

Training Dataset:  86111.9 Test Dataset:  36905.1


In [7]:
c = list(zip(Left_images, Right_images, Fx, Fy, U0, V0, Baseline, Disparity, Tx, Ty, Tz, Pitch, X, Y, Z))
random.shuffle(c)
Left_images, Right_images, Fx, Fy, U0, V0, Baseline, Disparity, Tx, Ty, Tz, Pitch, X, Y, Z = zip(*c)
Left_images, Right_images, Fx, Fy, U0, V0, Baseline, Disparity, Tx, Ty, Tz, Pitch, X, Y, Z = list(Left_images), list(Right_images), list(Fx), list(Fy), list(U0), list(V0), list(Baseline), list(Disparity), list(Tx), list(Ty), list(Tz), list(Pitch), list(X), list(Y), list(Z)



In [8]:
path = "../Data/"
np.save(path+"li.npy",Left_images)
np.save(path+"ri.npy",Right_images)
np.save(path+"fx.npy",Fx)
np.save(path+"fy.npy",Fy) 
np.save(path+"u0.npy",U0) 
np.save(path+"v0.npy",V0) 
np.save(path+"b.npy",Baseline)
np.save(path+"d.npy",Disparity) 
np.save(path+"tx.npy",Tx) 
np.save(path+"ty.npy",Ty) 
np.save(path+"tz.npy",Tz) 
np.save(path+"p.npy",Pitch)
np.save(path+"x.npy",X)
np.save(path+"y.npy",Y) 
np.save(path+"z.npy",Z)

# Real Data


In [7]:
from camera import *

left_imgs = []

right_imgs = []

disparity = []

camera_x = []
camera_baseline = []
camera_y = []
camera_z = []
camera_pitch = []
camera_fx = []
camera_fy = []
camera_u0 = []
camera_v0 = []

x_world = []
y_world = []
z_world = []

test_left_img_paths = []
test_disp_img_paths = []
test_cam_img_paths = []

cam = CIsoCamera()

data_path = "/home/talha/Documents/Tsinghua-Daimler Dataset/Dataset/Scripts/tsinghuaDaimlerScripts/Data/"
test_disparity_img_path = data_path+"disparity/test/tsinghuaDaimlerDataset/"
test_leftImg8bit_img_path = data_path+"leftImg8bit/test/tsinghuaDaimlerDataset/"
test_camera_path = data_path+"camera/test/tsinghuaDaimlerDataset/"

for i in os.listdir(test_disparity_img_path):
   
    test_disp_img_paths.append(test_disparity_img_path+i)
    
    test_left_img_paths.append(test_leftImg8bit_img_path+i.split(".")[0][:-9]+"leftImg8bit.png")
    
    test_cam_img_paths.append(test_camera_path+i.split(".")[0][:-9]+"camera.json")
    


for i in range(len(test_disp_img_paths)):

    
        
    dispImg = Image.open(test_disp_img_paths[i])
    dispNp = dispImg.resize((112,112), Image.ANTIALIAS)
    dispNp = np.array(dispNp,dtype=np.float)
    dispNp = ( dispNp - 1. ) / 256.
    disp = dispNp[5,5] + 1
    
    disparity.append(disp)
        
    leftImg = Image.open(test_left_img_paths[i])
    leftImg = leftImg.resize((112,112), Image.ANTIALIAS)
    left_imgs.append(np.array(leftImg))
    dispImg = Image.open(test_disp_img_paths[i])
    dispNp = dispImg.resize((112,112), Image.ANTIALIAS)
    dspimg = Image.new("RGB", dispNp.size)
    dspimg.paste(dispNp)
    rightImg = np.array(leftImg) - np.array(dspimg)
    right_imgs.append(np.array(rightImg))
        
    cam.loadFromJson(test_cam_img_paths[i])
        
    camera_x.append(cam.extrinsic.x)
    camera_baseline.append(cam.extrinsic.baseline)
    camera_y.append(cam.extrinsic.y)
    camera_z.append(cam.extrinsic.z)
    camera_pitch.append(cam.extrinsic.pitch)
    camera_fx.append(cam.intrinsic.fx)
    camera_fy.append(cam.intrinsic.fy)
    camera_u0.append(cam.intrinsic.u0)
    camera_v0.append(cam.intrinsic.v0)
        
        
    u = 5
    v = 5
    
        
    
       
    xCam = (cam.intrinsic.fx * cam.extrinsic.baseline) / disp
    yCam = - (xCam / cam.intrinsic.fx) * (u - cam.intrinsic.u0)
    zCam = (xCam / cam.intrinsic.fy) * (cam.intrinsic.v0 - v)

    yWorld = yCam + cam.extrinsic.y
    xWorld = xCam * math.cos(cam.extrinsic.pitch) + zCam*math.sin(cam.extrinsic.pitch) + cam.extrinsic.x
    zWorld = - xCam * math.sin(cam.extrinsic.pitch) + zCam*math.cos(cam.extrinsic.pitch) + cam.extrinsic.z
        
    x_world.append(xWorld)
    y_world.append(yWorld)
    z_world.append(zWorld)
        
        

        

In [10]:
print ("Training Dataset: ",len(left_imgs)*0.7, "Test Dataset: ", len(left_imgs)*0.3)

Training Dataset:  2039.8 Test Dataset:  874.1999999999999


In [8]:
c = list(zip(left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world))
random.shuffle(c)
left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world = zip(*c)
left_imgs, right_imgs, camera_fx, camera_fy, camera_u0, camera_v0, camera_baseline, disparity, camera_x, camera_y, camera_z, camera_pitch, x_world, y_world, z_world = list(left_imgs), list(right_imgs), list(camera_fx), list(camera_fy), list(camera_u0), list(camera_v0), list(camera_baseline), list(disparity), list(camera_x), list(camera_y), list(camera_z), list(camera_pitch), list(x_world), list(y_world), list(z_world)



In [9]:
np.save("lir.npy",left_imgs)
np.save("rir.npy",right_imgs)
np.save("fxr.npy",camera_fx)
np.save("fyr.npy",camera_fy) 
np.save("u0r.npy",camera_u0) 
np.save("v0r.npy",camera_v0) 
np.save("br.npy",camera_baseline)
np.save("dr.npy",disparity) 
np.save("txr.npy",camera_x) 
np.save("tyr.npy",camera_y) 
np.save("tzr.npy",camera_z) 
np.save("pr.npy",camera_pitch)
np.save("xr.npy",x_world)
np.save("yr.npy",y_world) 
np.save("zr.npy",z_world)